# AUTOCORRECT MODEL
- IDENTIFY MISSPELLED WORDS. "but without context..."
- MINIMIZE EDIT DISTANCES.
- CALCULATE PROBABILITIES OF THE CORRECT WORD.
-
![Alt text](../../figures/iamlearning.png)
- Get a word count given a corpus
- Get a word probability in the corpus 
- Manipulate strings 
- Filter strings 
- Implement Minimum edit distance to compare strings and to help find the optimal path for the edits. 
- Understand how dynamic programming works

## 1. IDENTIFY A MISSPELLED WORD
- When string word is NOT in dictionary. -> Spelling Error.
- Context error (later in jupyter notebook)

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import random # Printing

from utils.nlp import read_file_create_vocabulary

In [2]:
# Set path to text file
text_file = "../../DATA/shakespeare.txt"

# Get vocabulary - Read a file and output each unique word and the num. times they appear.
vocabulary = read_file_create_vocabulary(text_file, probabilities = False, information=True)

print(
    f"\nWords | Number of times\n{list(vocabulary.items())[0:3]}"
)

10 words:
['those', 'valour', 'likes', 'admirable', 'scorn', 'bertram', 'rational', 'charge', '85', 'dimpled']
There are 6116 unique words.

Words | Number of times
[('o', 157), ('for', 474), ('a', 757)]


## 2. MINIMUM EDIT DISTANCE - FIND STRINGS "n" EDIT DISTANCE AWAY
- Edit: Operation performed on a string to change it into another string.
- Minimum Edit distance: Minimum number of operations away one string is from another.
- Types of Edit:
- - Insert. ("to", "top", "too")
- - Delete. ("hat", "ha", "at")
- - Switch to adjacents. ("eta", "eat", "tea") BUT NO "ate" (change only ADJACENT).
- - Replace. ("jaw", "jar", "paw")
-
-
- n_edit distance is usually between 1 and 3.
-
- MINIMUM EDIT DISTANCE:
- - Allows to implement:
- - + SPELLING CORRECTION
- - + DOCUMENT SIMILARITY
- - + MACHINE TRANSLATION
- - + DNA SEQUENCING
##### Each EDIT can have different COST.
- These costs will give us the edit distance.
![Alt text](../../figures/editminimumdist.png)
##### Notice that to know distance from p -> s, we need to know the cell above (insertion, from # -> s), the cell to the left (deletion, from p -> #) and the adjacent upper left (replace).
![Alt text](../../figures/dynamicprogramming.png)
#### LEVENSHTEIN DISTANCE (Insert:1, Delte:1, Replace:2 Cost)
- Use BACKSTRACE "pointer that tells where does the path come from" for strings alignment.

In [ ]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
        
    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(0,m+1): # Replace None with the proper range
        D[row,0] = row*ins_cost
        
    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(0,n+1): # Replace None with the proper range
        D[0,col] = col*ins_cost
        
    # Loop through row 1 to row m, both inclusive
    for row in range(1,m+1):
        # Loop through column 1 to column n, both inclusive
        for col in range(1,n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            
            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            if source[row-1]==target[col-1]: # Replace None with a proper comparison
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row,col] = min(D[row-1,col-1]+r_cost,D[row-1,col]+del_cost,D[row,col-1]+ins_cost)
            
    # Set the minimum edit distance with the cost found at row m, column n 
    med = D[m,n]
    
    return D, med

## 3. FILTER CANDIDATES
- Consider only real words. After applying step2. We may need to compare the results with the dictionary.
- Similar to step 1.
- This time, if the string does not appear, remove it as candidate.

## 4. CALCULATE WORD PROBABILITIES
- Once we have:
- Word | num. times it appears in corpus.
-
- We can calculate the probabilities of each word such as:
- P(word_i) = Number of times word_i appears / Total number of words not unique.
-

In [4]:
# Set path to text file
text_file = "../../DATA/shakespeare.txt"

# Get vocabulary - Read a file and output each unique word and the num. times they appear.
vocabulary = read_file_create_vocabulary(text_file, probabilities = True, information=True)

print(
    f"\nWords | Probability\n{list(vocabulary.items())[0:3]}"
)

10 words:
['alters', 'golden', 'puissant', 'fading', 'receivest', 'supposes', 'gave', 'decays', 'action', 'rare']
There are 6116 unique words.

Words | Probability
[('o', 0.0029283396127877045), ('for', 0.008840974372365426), ('a', 0.01411944641325027)]


## AUTOCORRECTOR

In [5]:
from utils.nlp import StringOperations
autocorrector = StringOperations()

### 1. Create possible corrections and get word with higher probability

In [10]:
autocorrector.get_corrections(  word="dir",
                                words_probs=vocabulary,
                                number_corrections=2)

[('die', 0.0005595553400231282), ('sir', 0.0021636139814227625)]

### 2. Dynammic Programming

$$\text{Initialization}$$

Note that the formula for $D[i,j]$ shown in the image is equivalent to:

\begin{align}
 \\
D[i,j] =min
\begin{cases}
D[i-1,j] + del\_cost\\
D[i,j-1] + ins\_cost\\
D[i-1,j-1] + \left\{\begin{matrix}
rep\_cost; & if src[i]\neq tar[j]\\
0 ; & if src[i]=tar[j]
\end{matrix}\right.
\end{cases}
\tag{5}
\end{align}

The variable `sub_cost` (for substitution cost) is the same as `rep_cost`; replacement cost.  We will stick with the term "replace" whenever possible.

So converting the source word **play** to the target word **stay**, using an input cost of one, a delete cost of 1, and replace cost of 2 would give you the following table:
<table style="width:20%">

  <tr>
    <td> <b> </b>  </td>
    <td> <b># </b>  </td>
    <td> <b>s </b>  </td>
    <td> <b>t </b> </td> 
    <td> <b>a </b> </td> 
    <td> <b>y </b> </td> 
  </tr>
   <tr>
    <td> <b>  #  </b></td>
    <td> 0</td> 
    <td> 1</td> 
    <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
 
  </tr>
  <tr>
    <td> <b>  p  </b></td>
    <td> 1</td> 
 <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
   <td> 5</td>
  </tr>
   
  <tr>
    <td> <b> l </b></td>
    <td>2</td> 
    <td>3</td> 
    <td>4</td> 
    <td>5</td> 
    <td>6</td>
  </tr>

  <tr>
    <td> <b> a </b></td>
    <td>3</td> 
     <td>4</td> 
     <td>5</td> 
     <td>4</td>
     <td>5</td> 
  </tr>
  
   <tr>
    <td> <b> y </b></td>
    <td>4</td> 
      <td>5</td> 
     <td>6</td> 
     <td>5</td>
     <td>4</td> 
  </tr>
  

</table>



In [7]:
source =  'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


#### After computing matrix using minimum edit distance, how would find the shortest path from the top left corner to the bottom right corner? 

In [8]:
# BACKTRACE
def back_trace(D):
    '''
    Input:
        D : a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
    Output:
        idx : a list of tuples (i,j) indices for shortest path
    '''
    source_l = D.shape[0]
    target_l = D.shape[1]
    
    idx = []
    
    # BFS
    i,j=source_l-1,target_l-1
    idx.append((i,j))
    while(i>0 and j>0):
        cur = D[i,j]
        candid = sorted([((i-1,j-1),D[i-1,j-1]),((i-1,j),D[i-1,j]),((i,j-1),D[i,j-1])],key = lambda x : x[1])
        candid = candid[0]
        idx.append(candid[0])
        i,j = candid[0]
        
    return idx

In [9]:
## Shows the path from bottom corner to top corner.
back_trace(matrix)

[(4, 4), (3, 3), (2, 2), (1, 1), (0, 0)]